In [1]:
import requests
import json
import plotly.express as px

import numpy as np

from datetime import datetime
import pandas as pd

# custom libraries
from data import Data
from util import *
from visualization import *

In [2]:
# Leemos api.json
with open('api.json') as file:
    api = json.load(file)
    clientID = api['IGDB']['clientID']
    clientSecret = api['IGDB']['clientSecret']

In [3]:
# Creamos objeto Data
data = Data(clientID = clientID, clientSecret = clientSecret)

In [4]:
# Extraemos datos de IGDB utilizando su API
data.extract(endpoints = ['games', 'game_engines', 'language_supports', 'languages', 'genres'], batches = 50, show_logs = True, keep_logs = True, save_csv = True)

In [5]:
# Leemos los datos extraidos
data.read_csvs(paths = ['games_data.csv', 'game_engines_data.csv', 'language_supports_data.csv', 'languages_data.csv', 'genres_data.csv'])

In [6]:
# Nos quedamos solamente con las columnas que nos interesan
data.filterColumns(columns = ['id', 'name', 'language_supports', 'game_engines', 'first_release_date', 'genres', 'category'], inplace = True)

In [7]:
# Columnas con las que se va a trabajar
data.main

,id,name,language_supports,game_engines,first_release_date,genres,category
0,35642,Dungeon Crawlers HD,"[76103, 76104, 76105]",NaN,1.433117e+09,"[12, 15, 16, 24, 32]",0
1,246925,Stickman and the Sword of Legends,"[678670, 678671, 678672]",NaN,1.682640e+09,"[31, 32]",0
2,245087,Cry Baby,NaN,NaN,6.836832e+08,[8],0
3,85450,Transformers Prime: The Game,NaN,NaN,NaN,NaN,0
4,95080,Dotra,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...
24995,192600,Hot Hentai Puzzle Vol.2,"[584643, 584644, 584645, 584646, 584647]",NaN,1.637539e+09,[32],0
24996,162114,Jigsaw Puzzle Pack: Pixel Puzzles Ultimate - V...,NaN,NaN,1.609805e+09,"[13, 15, 32]",1
24997,108364,War War War: Smiles vs. Ghosts,[172527],NaN,1.535328e+09,"[31, 32]",0
24998,112305,A Timeless Carol,NaN,NaN,NaN,NaN,0


In [8]:
# Se crean columnas en el dataframe de language_supports, una por cada tipo de language_support_type
# Excluimos el language_support para la interfaz porque no nos parece un dato interesante
data.splitColumn(data_frame = 'language_supports',
                column = 'language',
                query_field = 'language_support_type',
                queries = [1,2],
                inplace = True
                 )

# Se crean dos columnas nuevas con supports para audio y subtitulos con la copia de la columna de language_supports
data.main['audio_language_supports'] = data.main['language_supports']
data.main['subtitles_language_supports'] = data.main['language_supports']

# Se modifican los id's de la tabla language_support por los id's de la tabla languages
data.parseLists(columns = ['audio_language_supports', 'subtitles_language_supports'],
                data_frames = ['language_supports', 'language_supports'],
                fields = ['language_1', 'language_2'],
                inplace = True
               )

# Se eliminan todos los np.nan que se encuentran dentro de listas
# Si la lista está compuesta completamente por np.nan, se sustituye por un np.nan
data.main['audio_language_supports'] = data.main['audio_language_supports'].apply(removeNaFromLists).apply(removeEmptyLists)
data.main['subtitles_language_supports'] = data.main['subtitles_language_supports'].apply(removeNaFromLists).apply(removeEmptyLists)

# Se cambian los id's de la tabla languages por los nombres de los lenguajes
data.parseLists(columns = ['audio_language_supports', 'subtitles_language_supports'],
                data_frames = ['languages', 'languages'],
                fields = ['name', 'name'],
                inplace = True
               )

In [10]:
# Convertimos timestamp en un objeto datetime
data.main['first_release_date'] = data.main['first_release_date'].apply(lambda x : datetime.fromtimestamp(x).strftime('%m-%d-%Y') if not pd.isna(x) else np.nan)

# Creamos columna con los meses
data.main['month_release'] = data.main['first_release_date'].apply(lambda x : x[:2] if not pd.isna(x) else np.nan)

# Creamos columna con los años
data.main['year_release'] = data.main['first_release_date'].apply(lambda x : x[-4:] if not pd.isna(x) else np.nan)

In [12]:
# Se cambian los id's de la tabla game_engines por los nombres de los motores
data.parseLists(columns = ['game_engines'],
                data_frames = ['game_engines'],
                fields = ['name'],
                inplace = True
               )

In [13]:
# Se cambian los id's de la tabla genres por los nombres de los géneros
data.parseLists(columns = ['genres'],
                data_frames = ['genres'],
                fields = ['name'],
                inplace = True
               )

In [14]:
# Se cambian los id's de las categorías por los nombres
categorys = diccionario = {0: 'main_game',
                           1: 'dlc_addon',
                           2: 'expansion',
                           3: 'bundle',
                           4: 'standalone_expansion',
                           5: 'mod',
                           6: 'episode',
                           7: 'season',
                           8: 'remake',
                           9: 'remaster',
                           10: 'expanded_game',
                           11: 'port',
                           12: 'fork',
                           13: 'pack',
                           14: 'update'}


data.main['category'] = data.main['category'].replace(categorys)

In [15]:
# Nos quedamos con las columnas procesadas
data.filterColumns(columns = ['id',
                              'name',
                              'game_engines',
                              'genres',
                              'category',
                              'audio_language_supports',
                              'subtitles_language_supports',
                              'month_release',
                              'year_release'
                             ],
                   inplace = True
                  )

In [16]:
# Datos procesados

data.main

,id,name,game_engines,genres,category,audio_language_supports,subtitles_language_supports,month_release,year_release
0,35642,Dungeon Crawlers HD,NaN,"[Role-playing (RPG), Strategy, Turn-based stra...",main_game,[English],[English],05,2015
1,246925,Stickman and the Sword of Legends,NaN,"[Adventure, Indie]",main_game,NaN,NaN,04,2023
2,245087,Cry Baby,NaN,[Platform],main_game,NaN,NaN,08,1991
3,85450,Transformers Prime: The Game,NaN,NaN,main_game,NaN,NaN,NaN,NaN
4,95080,Dotra,NaN,NaN,main_game,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
24995,192600,Hot Hentai Puzzle Vol.2,NaN,[Indie],main_game,NaN,NaN,11,2021
24996,162114,Jigsaw Puzzle Pack: Pixel Puzzles Ultimate - V...,NaN,"[Simulator, Strategy, Indie]",dlc_addon,NaN,NaN,01,2021
24997,108364,War War War: Smiles vs. Ghosts,NaN,"[Adventure, Indie]",main_game,NaN,NaN,08,2018
24998,112305,A Timeless Carol,NaN,NaN,main_game,NaN,NaN,NaN,NaN


In [17]:
key = api['Airtable']['key']
app = api['Airtable']['app']
tbls = api['Airtable']['tbls']

In [18]:
loadToAirtable(key, app, tbls, data.main) #tarda aprox 4 horas y media

In [19]:
dataframe = extractFromAirtable(key, app, tbls) #tarda menos de media hora

In [21]:
languages_fig(dataframe)

In [22]:
releases_fig(dataframe)

In [23]:
engines_fig(dataframe)

In [24]:
engines_years_fig(dataframe)

In [25]:
genres_categories_fig(dataframe)

/home/cobayaculera/yes/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/cobayaculera/yes/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

